In [2]:
%run Heterologous_insert.ipynb

In [3]:
model.compartments

{'ce': 'cell envelope',
 'c': 'cytoplasm',
 'e': 'extracellular',
 'm': 'mitochondrion',
 'n': 'nucleus',
 'p': 'peroxisome',
 'er': 'endoplasmic reticulum',
 'g': 'Golgi',
 'lp': 'lipid particle',
 'v': 'vacuole',
 'erm': 'endoplasmic reticulum membrane',
 'vm': 'vacuolar membrane',
 'gm': 'Golgi membrane',
 'mm': 'mitochondrial membrane'}

In [4]:
print(model.objective)

Maximize
1.0*r_2111 - 1.0*r_2111_reverse_58b69


In [5]:
model.medium

{'r_1654': 1000.0,
 'r_1714': 1.0,
 'r_1832': 1000.0,
 'r_1861': 1000.0,
 'r_1992': 1000.0,
 'r_2005': 1000.0,
 'r_2020': 1000.0,
 'r_2049': 1000.0,
 'r_2060': 1000.0,
 'r_2100': 1000.0,
 'r_4593': 1000.0,
 'r_4594': 1000.0,
 'r_4595': 1000.0,
 'r_4596': 1000.0,
 'r_4597': 1000.0,
 'r_4600': 1000.0}

     'r_1654': Ammonium
     'r_1714': D-glucose
     'r_1832': H+
     'r_1861': Iron(2+)
     'r_1992': Oxygen
     'r_2005': Phosphate
     'r_2020': Potassium
     'r_2049': Sodium
     'r_2060': Sulphate
     'r_2100': H20
     'r_4593': Chloride
     'r_4594': Cu2(+)
     'r_4595': Mn(2+)
     'r_4596': Zn(2+)
     'r_4597': Mg(2+)
     'r_4600': Ca(2+)

Not sure if we want to change the medium of growth - but here's a list

### INSTEAD - Go check the metabolic activity 

In [6]:
model.objective = model.reactions.DM_Eicosapentaenoic_Acid
medium = model.medium
with model:
    medium['r_1714'] = 20
    model.medium = medium
    EPA_production = model.optimize().objective_value
    print('Maximum productivity =', EPA_production, 'mmol/gDW*h')
    maximum_yield = EPA_production / (-1*(model.reactions.get_by_id('r_1714').flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol-EPA/mmol-glc')
    print('Objective of model is =', model.objective)


Maximum productivity = 3.5820895522388043 mmol/gDW*h
Maximum theoretical yield = 0.1791044776119402 mmol-EPA/mmol-glc
Objective of model is = Maximize
1.0*DM_Eicosapentaenoic_Acid - 1.0*DM_Eicosapentaenoic_Acid_reverse_e7faa


### Lets try to optimize the medium

According to [https://onlinelibrary.wiley.com/doi/epdf/10.1002/ejlt.200700224], the optimal medium for *L. starkeyi* is: 
mixed  sugar 73.3 g/L  (glucose  48.9 g/L,  xylose  24.4 g/L),  yeast  extract 7.9 g/L, FeSO 44.0 mg/L, KH2PO 47.0 g/L, Na2HPO 42.0 g/L, MgSO4-7H2O 1.0 g/L, and NH4Cl 0.5 g/L.

In [7]:
OPTmedium = model.medium

OPTmedium["r_1718"] = 100 #Xylose exchange
OPTmedium["r_1714"] = 100 #Glucose exchange
OPTmedium["r_2020"] = 1 #Potassium exchange
OPTmedium["r_1861"] = 1#Fe2+
OPTmedium["r_2049"] = 1 #Sodium
OPTmedium["r_1654"] = 1 #Ammonium
OPTmedium["r_1832"] = 10 #H+
OPTmedium["r_1992"] = 10 #Oxygen
OPTmedium["r_2005"] = 10 #Phosphate
OPTmedium["r_2060"] = 10 #Sulphate
OPTmedium["r_2100"] = 5 #H20
OPTmedium["r_4593"] = 10 #Chloride
OPTmedium["r_4594"] = 10 #Cu2+
OPTmedium["r_4595"] = 10 #Mn2+
OPTmedium["r_4596"] = 10 #Zn2+
OPTmedium["r_4597"] = 10 #Mg2+
OPTmedium["r_4600"] = 10 #Ca2+

# Maybe some amino acids should also be considered - All have exchange reactions in the model

model.medium = OPTmedium

model.objective = model.reactions.DM_Eicosapentaenoic_Acid
medium = model.medium
with model:
    medium['r_1714'] = 20
    model.medium = medium
    EPA_production = model.optimize().objective_value
    print('Maximum productivity =', EPA_production, 'mmol/gDW*h')
    maximum_yield = EPA_production / (-1*(model.reactions.get_by_id('r_1714').flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol-EPA/mmol-glc')
    print('Objective of model is =', model.objective)

Maximum productivity = 10.000000000000005 mmol/gDW*h
Maximum theoretical yield = 0.5000000000000002 mmol-EPA/mmol-glc
Objective of model is = Maximize
1.0*DM_Eicosapentaenoic_Acid - 1.0*DM_Eicosapentaenoic_Acid_reverse_e7faa


In [8]:
model.objective = model.reactions.get_by_id('r_2111')
solution = model.optimize()
print('Maximum theoretical biomass productivity:', solution.fluxes['r_2111'], '/h') 

model.objective = model.reactions.DM_Eicosapentaenoic_Acid
EPA_prod = model.optimize().objective_value
print('Maximum theoretical productivity of EPA:', EPA_prod, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.19717774256772388 /h
Maximum theoretical productivity of EPA: 9.99999999999998 mmol/gDW*h


In [9]:
solution.fluxes['r_2111']

0.19717774256772388

## Find the optimal medium for the model by setting bounds and optimizing the model

In [11]:
model.reactions.get_by_id('r_2111')

Reaction identifier,r_2111
Name,growth
Memory address,0x17528ba90
Stoichiometry,s_0450[c] --> biomass [cytoplasm] -->
GPR,
Lower bound,0.0
Upper bound,1000.0
